# Proyek Pertama Machine Learning Operations (MLOps) Dicoding menggunakan Dataset Plant Growth
by Tasya Putri Aliya

## Import Libraries

Melakukan import libraries yang akan digunakan dalam proyek ini

In [272]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

membuat directory dan path

In [273]:
PIPELINE_NAME = "tasyaputrialiya-pipeline"
SCHEMA_PIPELINE_NAME = "tasyaputrialiya-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

Membuat directory untuk data

In [274]:
DATA_ROOT = "data"

Membuat variabel untuk interactive_context

In [275]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Ingestion

proses mengambil, mempersiapkan, dan memasukkan data dari berbagai sumber ke dalam sistem atau platform yang akan digunakan untuk analisis, pemrosesan, atau penyimpanan lebih lanjut atau disebut pipeline

### ExampleGen

Code di bawah ini akan melakukan pembagian data antara data train dengan data test dengan perbandingan 80% training-20% testing dan menggunakan example_gen

In [276]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

Menggunakan interactive context untuk menampilkan hasil split contoh data

In [277]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### StatisticGen

Kode di bawah digunakan untuk memmbuat statistic summary mengguankan **StatisticGen** dengan input yang telah diproses oleh **ExampleGen**. Dapat dilihat hasil statistik dari data

In [278]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

Di bawah ini merupakan summary statistic dari data

In [279]:
interactive_context.show(statistics_gen.outputs["statistics"])

### SchemaGen

Kode di bawah ini digunakan untuk membuat data schema dari statistik data yang sudah dibuat sebelumnya. Inputnya adalah summary statistic

In [280]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

Berikut merupakan tampilan dari data schema yang dimiliki oleh data. Terdapat **feature name** dan **domain name** pada schema di bawah. **Feature name** merupakan feature atau kolom yang dimiliki data dan **domain name** merupakan kategori yang dimiliki oleh data yang berbentuk String categorical

In [281]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Fertilizer_Type',STRING,required,,'Fertilizer_Type'
'Growth_Milestone',INT,required,,-
'Humidity',FLOAT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'
'Sunlight_Hours',FLOAT,required,,-
'Temperature',FLOAT,required,,-
'Water_Frequency',STRING,required,,'Water_Frequency'


,Values
Domain,
'Fertilizer_Type',"'chemical', 'none', 'organic'"
'Soil_Type',"'clay', 'loam', 'sandy'"
'Water_Frequency',"'bi-weekly', 'daily', 'weekly'"


### ExampleValidator

Kode di bawah ini berfungsi untuk mengidentifikasi anomali yang terdapat pada dataset. Komponen yang digunakan adalah **ExampleValidator** dan input yang dibutuhkan adalah hasil dari **StatisticGen** dan **SchemaGen**

In [282]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

Di bawah ini merupakan hasil dari pengidentifikasian anomali. Dataset ini tidak memiliki anomali

In [283]:
interactive_context.show(example_validator.outputs['anomalies'])

## Data Preprocessing

Tahap preprocessing merupakan tahapan lanjutan dari data ingestion yang berfungsi untuk membersihkan, mengubah, dan menyiapkan data mentah menjadi format yang lebih sesuai

### Transform

Mendefinisikan nama modul Transform

In [284]:
TRANSFORM_MODULE_FILE = "plantgrowth-transform.py"

Membuat code untuk melakukan transform data menggunakan beberapa function, yaitu **transformed_name()** untuk mengubah nama fitur yang telah ditransform dan **preprocessing_fn()** untuk mengubah feature ke bentuk lowercase dan memastikan tipe data.

In [285]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "Growth_Milestone"
FEATURE_KEYS = ['Sunlight_Hours', 'Temperature', 'Humidity']

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features

    Args:
        inputs: map from feature keys to raw features.

    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}
    
    # Transform numerical features
    for key in FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.scale_to_z_score(inputs[key])
    
    # Transform label
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Overwriting plantgrowth-transform.py


Mendefinisikan komponen transform dengan input dari ExampleGen, SchemaGen, dan module transform sebelumnya. Hasil transformasi dapat dilihat di bawah

In [286]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

INFO:tensorflow:Assets written to: pipelines\tasyaputrialiya-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\95d168bbe1f04178a545853ded5fb3a0\assets


INFO:tensorflow:Assets written to: pipelines\tasyaputrialiya-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\95d168bbe1f04178a545853ded5fb3a0\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Tahap Pengembangan Model

Pada tahap ini akan dilakukan pengembangan model atau pembuatan model yang dimulai dengan melakukan train model

Mendefinisikan modul trainer

In [287]:
TRAINER_MODULE_FILE = "plantgrowth_trainer.py"

Membuat isi dari module file trainer yang di dalamnya berisi beberapa function, yaitu:
1. **transformed_name()** untuk melakukan transformasi nama
2. **gzip_reader_fn()** untuk memuat data dalam TFRecord
3. **input_fn()** untuk membuat transformed_feature yang dihasilkan komponen transform
4. **model_builder()** untuk membuat arsitektur model 
5. **_get_serve_tf_examples_fn()** untuk menjalankan tahapan preprocessing
6. **run_fn()** untuk menjalankan proses training

In [288]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "Growth_Milestone"
FEATURE_KEYS = ['Sunlight_Hours', 'Temperature', 'Humidity']

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=64) -> tf.data.Dataset:
    """Get post_transform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY))
    return dataset

def model_builder():
    """Build machine learning model"""
    inputs = {transformed_name(key): tf.keras.Input(shape=(1,), name=transformed_name(key), dtype=tf.float32) for key in FEATURE_KEYS}
    x = layers.Concatenate()(list(inputs.values()))
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    
    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # Get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=10)
    
    # Build the model
    model = model_builder()
    
    # Train the model
    model.fit(
        x=train_set,
        validation_data=val_set,
        callbacks=[tensorboard_callback, es, mc],
        steps_per_epoch=1000, 
        validation_steps=1000,
        epochs=10
    )

    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting plantgrowth_trainer.py


Mendefinisikan komponen trainer yang memiliki input module file trainer sebelumnya, ExampleGen, SchemaGen, transform_graph, train_args, dan eval_args

In [289]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000)
)
interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Sunlight_Hours_xf (InputLayer)  [(None, 1)]         0           []                               
                                                                                                  
 Temperature_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 Humidity_xf (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 3)            0           ['Sunlight_Hours_xf[0][0]',      
                                                                  'Temperature_xf[0][0]',   


Epoch 1: val_binary_accuracy improved from -inf to 0.48780, saving model to pipelines\tasyaputrialiya-pipeline\Trainer\model\6\Format-Serving


INFO:tensorflow:Assets written to: pipelines\tasyaputrialiya-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\tasyaputrialiya-pipeline\Trainer\model\6\Format-Serving\assets


1000/1000 [==============================] - 9s 5ms/step - loss: 0.6312 - binary_accuracy: 0.6237 - val_loss: 0.7604 - val_binary_accuracy: 0.4878
INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\tasyaputrialiya-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\tasyaputrialiya-pipeline\Trainer\model\6\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Tuner

Tahapan ini dilakukan untuk mendapatkan hyperparameter yang terbaik bagi model dengan dataset yang saat ini digunakan

Mendefinisikan module file tuner

In [290]:
TUNER_MODULE_FILE = "plantgrowth_tuner.py"

Membuat isi dari module file tuner dengan menggunakan function, yaitu:
1. **transformed_name()** untuk mentransformasi nama 
2. **gzip_reader_fn()** untuk memuat data dalam TFRecord
3. **input_fn()** untuk mengambil input
4. **model_builder** untuk membuat model yang akan digunakan dalam tuner
5. **tuner_fn()** untuk menjalankan tuner

In [291]:
%%writefile {TUNER_MODULE_FILE}
from typing import NamedTuple, Dict, Any, Text
from tfx.components.trainer.fn_args_utils import FnArgs
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
import keras_tuner as kt
from keras_tuner.engine.base_tuner import BaseTuner
import tensorflow_transform as tft
import tensorflow as tf

TunerFnResult = NamedTuple('TunerFnResult', [('tuner', BaseTuner), ('fit_kwargs', Dict[Text, Any])])

LABEL_KEY = "Growth_Milestone"
FEATURE_KEYS = ['Sunlight_Hours', 'Temperature', 'Humidity']


def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs=None, batch_size=64):
    """Get post_transform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY))
    return dataset

def model_builder(hp):
    """Build machine learning model with hyperparameters."""
    inputs = {transformed_name(key): tf.keras.Input(shape=(1,), name=transformed_name(key), dtype=tf.float32) for key in FEATURE_KEYS}
    x = layers.Concatenate()(list(inputs.values()))
    x = layers.Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32), activation='relu')(x)
    x = layers.Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1))(x)
    x = layers.Dense(units=hp.Int('units_2', min_value=16, max_value=64, step=16), activation='relu')(x)
    x = layers.Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1))(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    
    return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    """Build the tuner using the KerasTuner API."""
    
    # Load the transformed data
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=10)

    # Define the hyperband tuner
    tuner = kt.Hyperband(
        model_builder,
        objective='val_binary_accuracy',
        max_epochs=10,
        factor=3,
        directory=fn_args.working_dir,
        project_name='kt_hyperband'
    )

    # Set fit arguments for the tuner
    early_stopping = EarlyStopping(monitor='val_binary_accuracy',  mode='max', min_delta=0.001, patience=5, verbose=1)

    fit_kwargs = {
        "callbacks": [early_stopping],
        'x': train_set,
        'validation_data': val_set,
        'steps_per_epoch': fn_args.train_steps,
        'validation_steps': fn_args.eval_steps
    }

    return TunerFnResult(tuner=tuner, fit_kwargs=fit_kwargs)



Overwriting plantgrowth_tuner.py


Membuat komponen dari tuner 

In [292]:
from tfx.components import Tuner
from tfx.proto import trainer_pb2

tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
)

interactive_context.run(tuner)

Trial 30 Complete [00h 00m 05s]
val_binary_accuracy: 0.707317054271698

Best val_binary_accuracy So Far: 0.707317054271698
Total elapsed time: 00h 01m 58s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\tasyaputrialiya-pipeline\.temp\7\kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
units_1: 96
dropout_1: 0.1
units_2: 48
dropout_2: 0.0
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.707317054271698
Trial summary
Hyperparameters:
units_1: 96
dropout_1: 0.30000000000000004
units_2: 32
dropout_2: 0.30000000000000004
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.6097561120986938
Trial summary
Hyperparameters:
units_1: 64
dropout_1: 0.30000000000000004
units_2: 32
dropout_2: 0.1
learning_rate: 0.001
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.5853658318519592
Trial summary
Hyperparameters:
units_1: 96
dropout_1: 0.30000000000000004
units_2: 32
dropout_2: 0.30000000000000004
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 

ExecutionResult(
    component_id: Tuner
    execution_id: 7
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

### Resolver

Resolver digunakan untuk membuat baseline model untuk digunakan dalam analisis dan validasi model. Hal ini dapat dilakukan dengan menggunakan **Resolver**. Kode di bawah ini akan menghasilkan komponen Resolver

In [293]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### Evaluator

Kode di bawah digunakan untuk membuat konfigurasi dalam mengevaluasi model menggunakan library TFMA

In [294]:
import tensorflow_model_analysis as tfma

# Ensure your label key matches the one in your data
label_key = "Growth_Milestone"  # Update this if your label key is different

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key=label_key)],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="AUC"),
                tfma.MetricConfig(class_name="FalsePositives"),
                tfma.MetricConfig(class_name="TruePositives"),
                tfma.MetricConfig(class_name="FalseNegatives"),
                tfma.MetricConfig(class_name="TrueNegatives"),
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": 0.5}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 0.0001},
                        ),
                    ),
                ),
            ]
        )
    ],
)

Mendefinisikan komponen evaluator yang membutuhkan input berupa ExampleGen, mmodel dari turner, baseline_model dari Resolver, konfigurasi evaluasi sebelumnya

In [295]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

Berikut merupakan hasil evaluasi dari code, didapatkan bahwa **loss dari model sebesar 0.76**

In [296]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

### Pusher

Komponen pusher ini akan menerima  input berupa model yang sudah ditrain, hasil evaluasi dari evaluator, dan argumen terkait serving file path

In [297]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/plantgrowth-model'))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

Kode di bawah digunakan untuk memastikan versi model yang digunakan oleh TF-Serving

In [298]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/plantgrowth-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1720703210'}]}
